In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# data import path

path  = r'C:\Users\Layzr\Desktop\School\Data Immersion Part 4\Achievement 4'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

In [4]:
# using groupby to find the mean of the department_id column with a customers order count

ords_prods_merge.groupby('department_id').agg({'customer_order_count': ['mean']})

,customer_order_count
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# The minimum changes from a 14.8 to a 15.2 
# The maximum changes from a 21.9 to a 22.9
# The biggest increase is 6 from a 15.3 to a 16.4
# The biggest decrease is 8 from a 16.4 to a 15.3
# However overall the numbers stay very similar

In [5]:
# creating the max_order column to find customer loyalty

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['customer_order_count'].transform(np.max)

C:\Users\Layzr\AppData\Local\Temp\ipykernel_18788\2981464315.py:3: FutureWarning: The provided callable <function max at 0x0000015FDF1371A0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['customer_order_count'].transform(np.max)


In [6]:
# defining a customers loyalty

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
# Finding the spending habits for different customer types

ords_prods_merge.groupby('loyalty_flag')['prices'].mean()

loyalty_flag
Loyal customer      10.386336
New customer        13.294670
Regular customer    12.495717
Name: prices, dtype: float64

# Loyal customers surprisingly have the lowest spending habits compared to New and Regular Customers with New Customers actually having the highest spending habits

In [10]:
# creating a spending flag

ords_prods_merge['user_average_spent'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\Layzr\AppData\Local\Temp\ipykernel_18788\245824298.py:3: FutureWarning: The provided callable <function mean at 0x0000015FDF137BA0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['user_average_spent'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [11]:
# defining high and low spenders

ords_prods_merge.loc[ords_prods_merge['user_average_spent'] < 10, 'spending_flag'] = 'Low spender'

In [12]:
ords_prods_merge.loc[ords_prods_merge['user_average_spent'] >= 10, 'spending_flag'] = 'High spender'

In [13]:
# checking frequency of the spending flag

ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [14]:
# creating a user purchase frequency flag

ords_prods_merge['user_medians'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

C:\Users\Layzr\AppData\Local\Temp\ipykernel_18788\1076528360.py:3: FutureWarning: The provided callable <function median at 0x0000015FDF26A840> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['user_medians'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [15]:
# defining the user frequency flag

ords_prods_merge.loc[ords_prods_merge['user_medians'] < 10, 'order_frequency_flag'] = 'Non-frequent customer'

In [16]:
ords_prods_merge.loc[ords_prods_merge['user_medians'] > 20, 'order_frequency_flag'] = 'Frequent customer'

In [17]:
ords_prods_merge.loc[(ords_prods_merge['user_medians'] > 10) & (ords_prods_merge['user_medians'] <= 20), 'order_frequency_flag'] = 'Regular Customer'

In [18]:
# checking the order frequency flag

ords_prods_merge[['user_id', 'days_since_prior_order', 'user_medians', 'order_frequency_flag']].head(10)

,user_id,days_since_prior_order,user_medians,order_frequency_flag
0,1,NaN,20.5,Frequent customer
1,1,NaN,20.5,Frequent customer
2,1,NaN,20.5,Frequent customer
3,1,NaN,20.5,Frequent customer
4,1,NaN,20.5,Frequent customer
5,1,15.0,20.5,Frequent customer
6,1,15.0,20.5,Frequent customer
7,1,15.0,20.5,Frequent customer
8,1,15.0,20.5,Frequent customer
9,1,15.0,20.5,Frequent customer


In [20]:
# changing all outlier prices to NaN

ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [21]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge_frequencies.pkl'))